In [5]:
from utils import get_spark_session
import os

spark = get_spark_session("iceberg_transactions_PySpark")

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- salary: double (nullable = true)
 |-- last_purchase: timestamp (nullable = true)
 |-- last_purchase_date: string (nullable = true)



In [2]:
TABLE_NAME = "nessie.dev.table_1"

df_test.writeTo(TABLE_NAME).createOrReplace()
print(f"Numero de linhas após criação: {spark.table(TABLE_NAME).count()}")
df_test.writeTo(TABLE_NAME).append()
print(f"Numero de linhas após append: {spark.table(TABLE_NAME).count()}")

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


Numero de linhas após criação: 100
Numero de linhas após append: 200


In [3]:
spark.sql("""
    SELECT * FROM nessie.dev.table_1.history;
""").show()

# spark.table("nessie.dev.table_1").show()

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2024-09-29 15:37:...|5565358984773854541|               NULL|              false|
|2024-09-29 15:37:...|5484199323931666385|5565358984773854541|              false|
|2024-09-29 15:38:...| 263071444238144259|               NULL|              false|
|2024-09-29 15:38:...|2128714391020491279| 263071444238144259|              false|
|2024-09-29 16:49:...| 463466068859629968|               NULL|              false|
|2024-09-29 16:49:...| 550167812940096497| 463466068859629968|              false|
|2024-09-29 17:10:...| 718049196851715350|               NULL|              false|
|2024-09-29 17:11:...|5445461876262848214| 718049196851715350|              false|
|2024-09-29 17:26:...|3086337887324965106|               NULL|               true|
|202

## Merge on Read and Copy-on-write

### When to use?

- **Copy-on-write**: Daily batch jobs where write speed is less a priority and read time is a high priority;
- **Merge-on-read (Position Deletes)**: Streaming and higher frequency batch (hourly) where write speed is very important with minor cost to read times. Regular compaction should be scheduled;
- **Merge-on-read (Equality Deletes)**: Very write intensive jobs where position deletes still aren't fast enough, much larger cost to reading so frequent compaction jobs will be necessary to manage;

In [8]:
spark.sql("""
CREATE TABLE nessie.dev.table_2 (
  id INT,
  name STRING,
  age INT,
  salary FLOAT,
  last_purchase_date DATE
) USING iceberg TBLPROPERTIES (
  'write.delete.mode'='copy-on-write',
  'write.update.mode'='copy-on-write',
  'write.merge.mode'='copy-on-write'
) PARTITIONED BY ( last_purchase_date ) 
""")
spark.sql("DESCRIBE EXTENDED nessie.dev.table_2").show(100)
spark.sql("SHOW TBLPROPERTIES nessie.dev.table_2").show(100)

DataFrame[]

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|                  id|                 int|   NULL|
|                name|              string|   NULL|
|                 age|                 int|   NULL|
|              salary|               float|   NULL|
|  last_purchase_date|                date|   NULL|
|# Partition Infor...|                    |       |
|          # col_name|           data_type|comment|
|  last_purchase_date|                date|   NULL|
|                    |                    |       |
|  # Metadata Columns|                    |       |
|            _spec_id|                 int|       |
|          _partition|struct<last_purch...|       |
|               _file|              string|       |
|                _pos|              bigint|       |
|            _deleted|             boolean|       |
|                    |                    |       |
|# Detailed 

In [4]:
spark.stop()